In [127]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('data/billboard-200.db')

In [128]:
albums_table = pd.read_sql_query('select * from albums', conn)[1:]

## Cleaning

Convert rank to integer, and add a column named 'power' to represent the value of each album's position on the ranking. Higher ranked means more power.

In [129]:
albums_table['rank'] = pd.to_numeric(albums_table['rank'], downcast='signed')
albums_table['power'] = 1 / albums_table['rank']
albums_table['date'] = pd.to_datetime(albums_table['date'])

In [130]:
albums_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573946 entries, 1 to 573946
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   id            573946 non-null  int64         
 1   date          573946 non-null  datetime64[ns]
 2   artist        573946 non-null  object        
 3   album         573946 non-null  object        
 4   rank          573946 non-null  int16         
 5   length        492547 non-null  float64       
 6   track_length  468867 non-null  float64       
 7   power         573946 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int16(1), int64(1), object(2)
memory usage: 31.7+ MB


In [131]:
albums_table.head()

,id,date,artist,album,rank,length,track_length,power
1,2,2019-01-19,A Boogie Wit da Hoodie,Hoodie SZN,1,20.0,185233.800000,1.000000
2,3,2019-01-19,21 Savage,I Am > I Was,2,15.0,211050.733333,0.500000
3,4,2019-01-19,Soundtrack,Spider-Man: Into The Spider-Verse,3,13.0,190866.384615,0.333333
4,5,2019-01-19,Meek Mill,Championships,4,19.0,219173.894737,0.250000
5,6,2019-01-19,Post Malone,beerbongs & bentleys,5,18.0,214113.611111,0.200000


In [138]:
# fill in missing artist names
albums_table[albums_table['album']=='Silhouette'] = albums_table[albums_table['album']=='Silhouette'].replace({'':'Kenny G'})
albums_table[albums_table['album']=='Roots Of Country Music (1965)'] = albums_table[albums_table['album']=='Roots Of Country Music (1965)'].replace({'':'Various Artists'})

## Organizing and Refactoring

Create two subtables; one to represent every album that appears in the rankings, and one represent each artist that appears.

In [139]:
# albums: aggregated data about every album
albums = (albums_table.groupby(['album', 'artist'])
                      .agg({'power': 'sum',
                            'rank' : ['count', 'mean'],
                            'date' : ['min', 'max'],
                            'length': 'first',
                            'track_length': 'first'})
         )

albums.columns = ['power_rank',
                  'num_appearances',
                  'average_rank',
                  'first_appearance',
                  'last_appearance',
                  'length',
                  'track_length']
albums = albums.reset_index()

In [142]:
# artists: aggregated data about every artist
artists = (albums_table.groupby('artist')
                       .agg({'power': 'sum',
                             'rank' : ['count', 'mean'],
                             'album': 'nunique',
                             'date' : ['min', 'max']})
          )

artists.columns = ['power_rank',
                   'num_appearances',
                   'average_rank',
                   'num_albums',
                   'first_appearance',
                   'last_appearance',]